
# Automatic battery box calculator


This code enables automated calculation of the type of box required and the type and quantity of filler material to be used for transporting the batteries.
The user only needs to enter some information about the box, such as battery status (A, B or C), quantity, weight and dimensions.

Finally, there is an option to export the data frame as an Excel file.

In [10]:
import pandas as pd
import math

# cargar el archivo de datos
Behälterzuordung = pd.read_excel('Behälterzuordung(reduced).xlsx', sheet_name='Sheet1')
Behälterzuordung['Zuladung'] = Behälterzuordung['Zuladung'].fillna(0)

# convertir columnas a tipo int
Behälterzuordung['B'] = Behälterzuordung['B'].astype('int64')
Behälterzuordung['Zuladung'] = Behälterzuordung['Zuladung'].astype('int64')

# obtener los inputs del usuario

# Frage den Benutzer nach dem Batterietyp
battery_type = input("Geben Sie den gewünschten Batterietyp ein (A, B oder C): ").upper()

# Überprüfe den eingegebenen Batterietyp
if battery_type == "A":
    # Berechnungen und Operationen für Batterietyp A
    # ...
    pass
elif battery_type == "B":
    # Berechnungen und Operationen für Batterietyp B
    # ...
    pass
elif battery_type == "C":
    # Berechnungen und Operationen für Batterietyp C
    # ...
    pass
else:
    print("Ungültiger Batterietyp. Bitte geben Sie A, B oder C ein.")
    
num_batteries = int(input('N° Batterien: '))
battery_weight = float(input('Nettogewicht Batterie: '))
box_height = float(input('Geben Sie die Höhe der Box in mm an:'))
box_width = float(input('Geben Sie die Breite der Box in mm an:'))
box_length = float(input('Geben Sie die Länge der Box in mm an:'))

# calcular el número de baterías que caben en cada caja
Behälterzuordung['Batteries per box'] = Behälterzuordung['Zuladung'] // battery_weight
#
# calcular nuero de cajas necesarias
#Behälterzuordung['Boxes needed']= 15 / Behälterzuordung['Batteries per box']


# obtener las cajas que cumplen con las condiciones requeridas y el tipo de batería
boxes = Behälterzuordung.loc[(Behälterzuordung['L'] >= box_height) &
                             (Behälterzuordung['B'] >= box_width) &
                             (Behälterzuordung['H'] >= box_length) &
                             (Behälterzuordung['Batterietyp'].str.contains(battery_type, case=False)), :]




#Vermiculite, inspirado de excel

#sizes
medida_a=boxes["L"]/int(box_height)
medida_b=boxes["B"]/int(box_width)
medida_c=boxes["H"]/int(box_length)

#redondeos
medida_a = medida_a.apply(lambda x: math.floor(x))
medida_b = medida_b.apply(lambda x: math.floor(x))
medida_c = medida_c.apply(lambda x: math.floor(x))



max_Anzahl_Batterien_Behälter= medida_a*medida_b*medida_c
#tipo A y B
Behälter_bei_Max_Auss = num_batteries / max_Anzahl_Batterien_Behälter
Behälter_bei_Max_Auss = Behälter_bei_Max_Auss.apply(lambda x: math.ceil(x))

boxes["Vermiculite_Ausschöpfung"] = ((boxes["L"]*boxes["B"]*boxes["H"]*Behälter_bei_Max_Auss)-(int(box_height)*int(box_width)*int(box_length)*num_batteries))/1000000

boxes["Anzahl_100l_Säcke_Vermiculite"]=boxes["Vermiculite_Ausschöpfung"]/100
boxes["Anzahl_100l_Säcke_Vermiculite"] = boxes["Anzahl_100l_Säcke_Vermiculite"].apply(lambda x: math.ceil(x))





# sorting Battery Type and its policy; Type C == 1 Box per batterey

if battery_type == "A":
    boxes["Anzahl_der_benötigen_Behälter"] = Behälter_bei_Max_Auss 

    
    
    #Calculating Zuladung for type A
    #making formula for Zuladung
    boxes["Vermiculite_Ausschöpfung_Zuladung"] = ((boxes["L"]*boxes["B"]*boxes["H"]*Behälter_bei_Max_Auss)-(int(box_height)*int(box_width)*int(box_length)*num_batteries))/1000000
    Zuladung_KG_je_Behälter=((battery_weight*num_batteries)+(boxes["Vermiculite_Ausschöpfung_Zuladung"]/10))/boxes["Anzahl_der_benötigen_Behälter"]
  
 #turning zualdung into DF
    Zuladung_KG_je_Behälter = pd.DataFrame(Zuladung_KG_je_Behälter)
    Zuladung_KG_je_Behälter = Zuladung_KG_je_Behälter.rename(columns={"0": "Nueva_Columna_KG"})
 
 #renaming column
    Zuladung_KG_je_Behälter = Zuladung_KG_je_Behälter.rename(columns={0: 'Zuladung_KG_je_Behälter'})

# Redondear la columna "Valores" a 2 decimales
    Zuladung_KG_je_Behälter = Zuladung_KG_je_Behälter.round(2)


# Perform an inner merge on the index -BOth DF must be in the same DF
    merged_df = pd.merge(Zuladung_KG_je_Behälter,boxes , left_index=True, right_index=True)



# Suponiendo que ya tienes un DataFrame llamado df con las columnas 'Zuladung_KG_je_Behälter' y 'Zuladung'

# Crear una nueva columna llamada 'Resultado' y establecerla inicialmente como ' ' (vacio)
    merged_df['Ergebnis'] = 'Gut aber Check '
    


# Aplicar la condición y asignar 'no bueno' si se cumple
    merged_df.loc[(merged_df['Zuladung_KG_je_Behälter'] >= merged_df['Zuladung']) |
                 (merged_df['Zuladung_KG_je_Behälter'] == merged_df['Zuladung'] - 1) |
                 (merged_df['Zuladung_KG_je_Behälter'] == merged_df['Zuladung'] + 1), 'Ergebnis'] = 'max. Zuladung des Behälters wird überschritten'

    
    #Reordenar las columnas 
    
         # Extraer la columna a mover
    columna = merged_df.pop('Ergebnis')

# Insertar la columna en la posición deseada
    posicion = 9  # Posición a la que deseas mover la columna
    merged_df.insert(posicion, 'Ergebnis', columna)
    
     # Extraer la columna a mover
    columna = merged_df.pop('Anzahl_der_benötigen_Behälter')

# Insertar la columna en la posición deseada
    posicion = 10  # Posición a la que deseas mover la columna
    merged_df.insert(posicion, 'Anzahl_der_benötigen_Behälter', columna)

    
         # Extraer la columna a mover
    columna = merged_df.pop('Zuladung_KG_je_Behälter')

# Insertar la columna en la posición deseada
    posicion = 6  # Posición a la que deseas mover la columna
    merged_df.insert(posicion, 'Zuladung_KG_je_Behälter', columna)
    
    boxes = merged_df.sort_values(by='Batteries per box', ascending=True)
    
    
    
    
    
elif battery_type == "B":
    
    # filtering weight of battery type B
    if battery_weight >=30:
        
        boxes["Anzahl_der_benötigen_Behälter"]=num_batteries
        
        # Calcular vermiculita para el tipo C

        max_Anzahl_Batterien_Behälter = medida_a * medida_b * medida_c

        boxes["Vermiculite Gesamt in l"] = ((boxes["L"] * boxes["B"] * boxes["H"] * boxes["Anzahl_der_benötigen_Behälter"]) - (int(box_height) * int(box_width) * int(box_length) * num_batteries)) / 1000000
        
        
        boxes["Anzahl 100l Säcke Vermiculite"]= boxes["Vermiculite Gesamt in l"] # type B > 30kg follows a different patron to calcultate the Zuladung in kg je Behälter
        
        boxes["Anzahl 100l Säcke Vermiculite"] = boxes["Anzahl 100l Säcke Vermiculite"] / 100

        boxes["Anzahl 100l Säcke Vermiculite"] = boxes["Anzahl 100l Säcke Vermiculite"].apply(lambda x: math.ceil(x))
        boxes["Batteries per box"] = num_batteries # Type C = box per battery

        boxes.drop('Vermiculite_Ausschöpfung', axis=1, inplace=True) # keep only Vermiculi for type c
        boxes.drop('Anzahl_100l_Säcke_Vermiculite', axis=1, inplace=True)
        
        
        # Ordenar las cajas filtradas
        boxes = boxes.sort_values(by='Batteries per box', ascending=True)
        
        # Calculating Zuladung for type  C

        
        Zuladung_KG_je_Behälter = ((battery_weight * num_batteries) + (boxes["Vermiculite Gesamt in l"] / 10)) / boxes["Anzahl_der_benötigen_Behälter"]
        
        # Turning Zuladung into a DataFrame
        Zuladung_KG_je_Behälter = pd.DataFrame(Zuladung_KG_je_Behälter)
        Zuladung_KG_je_Behälter = Zuladung_KG_je_Behälter.rename(columns={0: 'Zuladung_KG_je_Behälter'})
        
        
        # Rounding the "Zuladung_KG_je_Behälter" column to 2 decimals
        Zuladung_KG_je_Behälter = Zuladung_KG_je_Behälter.round(2)
        
        # Perform an inner merge on the index - Both DF must be in the same DF
        merged_df = pd.merge(Zuladung_KG_je_Behälter, boxes, left_index=True, right_index=True)
        
        
        # Create a new column called 'Ergebnis' and initially set it as 'Gut aber Check'
        merged_df['Ergebnis'] = 'Gut aber Check'
        
        # Apply the condition and assign 'max. Zuladung des Behälters wird überschritten' if condition is met
        
        merged_df.loc[(merged_df['Zuladung_KG_je_Behälter'] >= merged_df['Zuladung']) |
                      (merged_df['Zuladung_KG_je_Behälter'] == merged_df['Zuladung'] - 1) |
                      (merged_df['Zuladung_KG_je_Behälter'] == merged_df['Zuladung'] + 1), 'Ergebnis'] = 'max. Zuladung des Behälters wird überschritten'
        
        # Reorder the columns
        # Extract the column to move
        columna = merged_df.pop('Ergebnis')
            
        # Insert the column at the desired position
        posicion = 9  # Position where you want to move the column
        merged_df.insert(posicion, 'Ergebnis', columna)
        
        # Extract the column to move
        columna = merged_df.pop('Anzahl_der_benötigen_Behälter')
        
        # Insert the column at the desired position
        posicion = 10  # Position where you want to move the column
        merged_df.insert(posicion, 'Anzahl_der_benötigen_Behälter', columna)
        
        # Extract the column to move
        columna = merged_df.pop('Zuladung_KG_je_Behälter')
        
        # Insert the column at the desired position
        posicion = 6  # Position where you want to move the column
        merged_df.insert(posicion, 'Zuladung_KG_je_Behälter', columna)
        
        
        
        # batteries per box in type B > 30kg and type C -> 1 battery = 1 box
        
        merged_df['Batteries per box'] = 1
        
        
        # Sort the boxes based on 'Batteries per box'
        boxes = merged_df.sort_values(by='Batteries per box', ascending=True)

        
    elif battery_weight <30:
        
        boxes["Anzahl_der_benötigen_Behälter"]=Behälter_bei_Max_Auss
        
        #Calculating Zuladung for type A
        #making formula for Zuladung
        

        boxes["Vermiculite_Ausschöpfung_Zuladung"] = ((boxes["L"]*boxes["B"]*boxes["H"]*Behälter_bei_Max_Auss)-(int(box_height)*int(box_width)*int(box_length)*boxes["Anzahl_der_benötigen_Behälter"]))/1000000

        Zuladung_KG_je_Behälter=((battery_weight*num_batteries)+(boxes["Vermiculite_Ausschöpfung_Zuladung"]/10))/boxes["Anzahl_der_benötigen_Behälter"]
        #turning zualdung into DF
        
        Zuladung_KG_je_Behälter = pd.DataFrame(Zuladung_KG_je_Behälter)
        Zuladung_KG_je_Behälter = Zuladung_KG_je_Behälter.rename(columns={"0": "Nueva_Columna_KG"})
        
        #renaming column  
        Zuladung_KG_je_Behälter = Zuladung_KG_je_Behälter.rename(columns={0: 'Zuladung_KG_je_Behälter'})
        
        

        # Redondear la columna "Valores" a 2 decimales
        Zuladung_KG_je_Behälter = Zuladung_KG_je_Behälter.round(2)  

        # Perform an inner merge on the index -BOth DF must be in the same DF
        merged_df = pd.merge(Zuladung_KG_je_Behälter,boxes , left_index=True, right_index=True)
        
        # Suponiendo que ya tienes un DataFrame llamado df con las columnas 'Zuladung_KG_je_Behälter' y 'Zuladung'
        
        # Crear una nueva columna llamada 'Resultado' y establecerla inicialmente como ' ' (vacio)
        
        merged_df['Ergebnis'] = 'Gut aber Check '
        
        
        # Aplicar la condición y asignar 'no bueno' si se cumple
        
        merged_df.loc[(merged_df['Zuladung_KG_je_Behälter'] >= merged_df['Zuladung']) |
                      (merged_df['Zuladung_KG_je_Behälter'] == merged_df['Zuladung'] - 1) |
                      (merged_df['Zuladung_KG_je_Behälter'] == merged_df['Zuladung'] + 1), 'Ergebnis'] = 'max. Zuladung des Behälters wird überschritten'
        
        

        #Reordenar las columnas 
        # Extraer la columna a mover
        columna = merged_df.pop('Ergebnis')
        
        # Insertar la columna en la posición deseada
        posicion = 9  # Posición a la que deseas mover la columna
        merged_df.insert(posicion, 'Ergebnis', columna)
        
        # Extraer la columna a mover
        columna = merged_df.pop('Anzahl_der_benötigen_Behälter')
        
        
        # Insertar la columna en la posición deseada
        posicion = 10  # Posición a la que deseas mover la columna
        merged_df.insert(posicion, 'Anzahl_der_benötigen_Behälter', columna)
        

        # Extraer la columna a mover
        columna = merged_df.pop('Zuladung_KG_je_Behälter')
        
        
        # Insertar la columna en la posición deseada
        posicion = 6  # Posición a la que deseas mover la columna
        merged_df.insert(posicion, 'Zuladung_KG_je_Behälter', columna)
        
        boxes = merged_df.sort_values(by='Batteries per box', ascending=True)
        
 
    
        
    else:
        print('Check the code line if battery_Type == "B" battery_weight >=30')

    
    
    
elif battery_type == "C":
    boxes = boxes.loc[boxes["Batterietyp"].str.contains('C')] # filtering Boxey by type C
    boxes["Anzahl_der_benötigen_Behälter"] = num_batteries
    
    # Calcular vermiculita para el tipo C
    max_Anzahl_Batterien_Behälter = medida_a * medida_b * medida_c

    boxes["Vermiculite Gesamt in l"] = ((boxes["L"] * boxes["B"] * boxes["H"] * num_batteries) - (int(box_height) * int(box_width) * int(box_length) * num_batteries)) / 1000000
             
    boxes["Anzahl 100l Säcke Vermiculite"]= boxes["Vermiculite Gesamt in l"] # type B > 30kg follows a different patron to calcultate the Zuladung in kg je Behälter
        
    boxes["Anzahl 100l Säcke Vermiculite"] = boxes["Anzahl 100l Säcke Vermiculite"] / 100

    boxes["Anzahl 100l Säcke Vermiculite"] = boxes["Anzahl 100l Säcke Vermiculite"].apply(lambda x: math.ceil(x))
    boxes["Batteries per box"] = num_batteries # Type C = box per battery

    boxes.drop('Vermiculite_Ausschöpfung', axis=1, inplace=True) # keep only Vermiculi for type c
    boxes.drop('Anzahl_100l_Säcke_Vermiculite', axis=1, inplace=True)
        
        
        # Ordenar las cajas filtradas
    boxes = boxes.sort_values(by='Batteries per box', ascending=True)
        
        # Calculating Zuladung for type  C

        
    Zuladung_KG_je_Behälter = ((battery_weight * num_batteries) + (boxes["Vermiculite Gesamt in l"] / 10)) / boxes["Anzahl_der_benötigen_Behälter"]
        
        # Turning Zuladung into a DataFrame
    Zuladung_KG_je_Behälter = pd.DataFrame(Zuladung_KG_je_Behälter)
    Zuladung_KG_je_Behälter = Zuladung_KG_je_Behälter.rename(columns={0: 'Zuladung_KG_je_Behälter'})
    
    # Rounding the "Zuladung_KG_je_Behälter" column to 2 decimals
    Zuladung_KG_je_Behälter = Zuladung_KG_je_Behälter.round(2)
    
    # Perform an inner merge on the index - Both DF must be in the same DF
    merged_df = pd.merge(Zuladung_KG_je_Behälter, boxes, left_index=True, right_index=True)
    
    # Create a new column called 'Ergebnis' and initially set it as 'Gut aber Check'
    merged_df['Ergebnis'] = 'Gut aber Check'
    
    
    # Apply the condition and assign 'max. Zuladung des Behälters wird überschritten' if condition is met
    
    merged_df.loc[(merged_df['Zuladung_KG_je_Behälter'] >= merged_df['Zuladung']) |
              (merged_df['Zuladung_KG_je_Behälter'] == merged_df['Zuladung'] - 1) |
              (merged_df['Zuladung_KG_je_Behälter'] == merged_df['Zuladung'] + 1), 'Ergebnis'] = 'max. Zuladung des Behälters wird überschritten'
    
    # Reorder the columns
    # Extract the column to move
    columna = merged_df.pop('Ergebnis')
    
    # Insert the column at the desired position
    posicion = 9  # Position where you want to move the column
    merged_df.insert(posicion, 'Ergebnis', columna)
    
    # Extract the column to move
    columna = merged_df.pop('Anzahl_der_benötigen_Behälter')
    
    # Insert the column at the desired position
    posicion = 10  # Position where you want to move the column
    merged_df.insert(posicion, 'Anzahl_der_benötigen_Behälter', columna)
    
    # Extract the column to move
    columna = merged_df.pop('Zuladung_KG_je_Behälter')
    
    # Insert the column at the desired position

    posicion = 6  # Position where you want to move the column
    merged_df.insert(posicion, 'Zuladung_KG_je_Behälter', columna)
    
    
    # batteries per box in type B > 30kg and type C -> 1 battery = 1 box
        
    merged_df['Batteries per box'] = 1
    
    # Sort the boxes based on 'Batteries per box'
    boxes = merged_df.sort_values(by='Batteries per box', ascending=True)
    
    #boxes.drop('Anzahl_100l_Säcke_Vermiculite', axis=1, inplace=True)
    

    
    #------------------------CALCULA EN CASO DE MAX ZULADUNG REACHED----------------------------------------------
    # si no hay max zuladung reached, a a dar error - ACHTUNG
    
    if boxes["Ergebnis"].str.contains('max. Zuladung des Behälters wird überschritten').any():
        
        alternative_boxes= boxes
        
        alternative_Anzahl_Batterien =  num_batteries #BATTEREINE TYPE C
        
        alternative_boxes["alternative_Anzahl_Batterien"] = alternative_Anzahl_Batterien  
        
        # new DF for alternatives  - ACTHUNG BATTERIEN TYPE C


        
        
        # sort daqta, showing only infos reagrding max Zualdung overpassed

        alternative_boxes = alternative_boxes.loc[alternative_boxes["Ergebnis"].str.contains('max')] 
        
        alternative_Anzahl_Batterien =  alternative_boxes["Zuladung"]/battery_weight
        
        alternative_boxes["alternative_Anzahl_Batterien"] = alternative_Anzahl_Batterien
        
        # alternative Zuladung in kg bei max. Ausschöpfung (wenn max. Zuladung überschritten)
        alternative_boxes["alternative_Zuladung_Auss_KG"] =  alternative_Anzahl_Batterien*battery_weight
        
        #alternative Anzahl der benötigten Behälter bei max. Ausschöpfung
        
        # mat ceil solo se aplica a numero finitos y no infinitos
        
        alternative_Anzahl_benötigten_Behälter_max_Ausschöpfung = num_batteries / alternative_Anzahl_Batterien
        alternative_boxes ["alternative_Anzahl_benötigten_Behälter_max_Ausschöpfung"] = alternative_Anzahl_benötigten_Behälter_max_Ausschöpfung.apply(lambda x: math.ceil(x) if not math.isinf(x) else x)
        
        # eliminar filas que contienen un 0 en alternative_Zuladung_Auss_KG (si la bateria pesa mas que el aguante de la caja = 0), asi evito numero infinitos o errores de formula
        alternative_boxes = alternative_boxes.drop(alternative_boxes[alternative_boxes["alternative_Zuladung_Auss_KG"] == 0].index)
        
        # alternative Anzahl der benötigen Behälter para tipo A y B en el c = n° batteries
        alternative_Anzahl_benötigen_Behälter = alternative_Anzahl_benötigten_Behälter_max_Ausschöpfung  
        alternative_boxes["alternative_Anzahl_benötigen_Behälter"] = alternative_Anzahl_benötigen_Behälter
        
        # Vermiculite Gesamt in l
        alternative_max_Vermiculite_Gesamt_1=((Behälterzuordung['L']* Behälterzuordung['B']* Behälterzuordung['H']*alternative_Anzahl_benötigen_Behälter)-(int(box_height) * int(box_width) * int(box_length) * num_batteries))/1000000
        alternative_boxes["alternative_max_Vermiculite_Gesamt_1"] = alternative_max_Vermiculite_Gesamt_1
        
        #alternative Zuladung in kg je Behälter (wenn max. Zuladung überschritten)
        
        alternative_Zuladung_kg_je_Behälter=((battery_weight*num_batteries)+(alternative_max_Vermiculite_Gesamt_1 / 10))/alternative_Anzahl_benötigen_Behälter
        alternative_boxes["alternative_Zuladung_kg_je_Behälter"] = alternative_Zuladung_kg_je_Behälter
        alternative_boxes["alternative_Zuladung_kg_je_Behälter"] = alternative_boxes["alternative_Zuladung_kg_je_Behälter"].apply(lambda x: math.ceil(x)) # round upd
        
        #Anzahl 100l Säcke Vermiculite
        alternative_Anzahl_100l_Säcke_Vermiculite = alternative_max_Vermiculite_Gesamt_1/100
        alternative_boxes["alternative_Anzahl_100l_Säcke_Vermiculite"] = alternative_Anzahl_100l_Säcke_Vermiculite
        alternative_boxes["alternative_Anzahl_100l_Säcke_Vermiculite"] = alternative_boxes["alternative_Anzahl_100l_Säcke_Vermiculite"].apply(lambda x: math.ceil(x)) # round upd
        
        #Menge Vermiculite je Behälter
        alternative_Menge_Vermiculite_Behälter =  alternative_max_Vermiculite_Gesamt_1 / alternative_Anzahl_benötigen_Behälter
        alternative_boxes["alternative_Menge_Vermiculite_Behälter"] = alternative_Menge_Vermiculite_Behälter
        
        # Lista de columnas a eliminar
        updated_alternative_boxes = alternative_boxes[['Hersteller',"Bezeichnung","L","B","H","Zuladung","Leergewicht","Batterietyp","Ergebnis","Batteries per box","alternative_Anzahl_Batterien","alternative_Zuladung_Auss_KG","alternative_Anzahl_benötigen_Behälter","alternative_Zuladung_kg_je_Behälter","alternative_max_Vermiculite_Gesamt_1","alternative_Anzahl_100l_Säcke_Vermiculite","alternative_Menge_Vermiculite_Behälter"]]
        
        pass
    else:
        pass  # No hacer nada


    
else:
    print("Something went wrong at variable Anzahl_der_benötigen_Behälter")

#-------------------------------- FORMULA AFTER MAX ZULADUNG REACHED FOR TYPE A  AND B -------------------



if (battery_type == "B" or battery_type == "A") and boxes["Ergebnis"].str.contains('max. Zuladung des Behälters wird überschritten').any():
    
    
    # new DF

    alternative_boxes= boxes

    # sort daqta, showing only infos reagrding max Zualdung overpassed

    alternative_boxes = alternative_boxes.loc[alternative_boxes["Ergebnis"].str.contains('max')] 
    alternative_Anzahl_Batterien =  alternative_boxes["Zuladung"]/battery_weight

    alternative_boxes["alternative_Anzahl_Batterien"] = alternative_Anzahl_Batterien = alternative_Anzahl_Batterien.apply(lambda x: math.floor(x)) #round down
    
    #If there is a 0 in alternative_boxes["alternative_Anzahl_Batterien"] its becasue the battery weight is higher than the box capacity, therefore the division and round dwons turns into 0, whihc will give an error in de code
    # Check if any row contains the number 0
    mask = alternative_boxes.eq(0).any(axis=1)
    
    # Replace the entire row with the message "Nettogewicht Batterie too high"
    alternative_boxes[mask] = 'Nettogewicht Batterie too high'
    
    
# alternative Zuladung in kg bei max. Ausschöpfung (wenn max. Zuladung überschritten)

    alternative_boxes["alternative_Zuladung_Auss_KG"] =  alternative_Anzahl_Batterien*battery_weight

    
    
    #alternative Anzahl der benötigten Behälter bei max. Ausschöpfung


    # mat ceil solo se aplica a numero finitos y no infinitos
    alternative_Anzahl_benötigten_Behälter_max_Ausschöpfung = num_batteries / alternative_Anzahl_Batterien
    alternative_boxes ["alternative_Anzahl_benötigten_Behälter_max_Ausschöpfung"] = alternative_Anzahl_benötigten_Behälter_max_Ausschöpfung.apply(lambda x: math.ceil(x) if not math.isinf(x) else x)

    # eliminar filas que contienen un 0 en alternative_Zuladung_Auss_KG (si la bateria pesa mas que el aguante de la caja = 0), asi evito numero infinitos o errores de formula

    alternative_boxes = alternative_boxes.drop(alternative_boxes[alternative_boxes["alternative_Zuladung_Auss_KG"] == 0].index)


    # alternative Anzahl der benötigen Behälter para tipo A y B en el c = n° batteries

    alternative_Anzahl_benötigen_Behälter = alternative_Anzahl_benötigten_Behälter_max_Ausschöpfung  
    alternative_boxes["alternative_Anzahl_benötigen_Behälter"] = alternative_Anzahl_benötigen_Behälter

    # Vermiculite Gesamt in l

    alternative_max_Vermiculite_Gesamt_1=((Behälterzuordung['L']* Behälterzuordung['B']* Behälterzuordung['H']*alternative_Anzahl_benötigen_Behälter)-(int(box_height) * int(box_width) * int(box_length) * num_batteries))/1000000
    alternative_boxes["alternative_max_Vermiculite_Gesamt_1"] = alternative_max_Vermiculite_Gesamt_1

    #alternative Zuladung in kg je Behälter (wenn max. Zuladung überschritten)

    alternative_Zuladung_kg_je_Behälter=((battery_weight*num_batteries)+(alternative_max_Vermiculite_Gesamt_1 / 10))/alternative_Anzahl_benötigen_Behälter
    alternative_boxes["alternative_Zuladung_kg_je_Behälter"] = alternative_Zuladung_kg_je_Behälter
    alternative_boxes["alternative_Zuladung_kg_je_Behälter"] = alternative_boxes["alternative_Zuladung_kg_je_Behälter"].apply(lambda x: math.ceil(x)) # round upd

    #Anzahl 100l Säcke Vermiculite

    alternative_Anzahl_100l_Säcke_Vermiculite = alternative_max_Vermiculite_Gesamt_1/100

    alternative_boxes["alternative_Anzahl_100l_Säcke_Vermiculite"] = alternative_Anzahl_100l_Säcke_Vermiculite

    alternative_boxes["alternative_Anzahl_100l_Säcke_Vermiculite"] = alternative_boxes["alternative_Anzahl_100l_Säcke_Vermiculite"].apply(lambda x: math.ceil(x)) # round upd

    #Menge Vermiculite je Behälter


    alternative_Menge_Vermiculite_Behälter =  alternative_max_Vermiculite_Gesamt_1 / alternative_Anzahl_benötigen_Behälter


    alternative_boxes["alternative_Menge_Vermiculite_Behälter"] = alternative_Menge_Vermiculite_Behälter

    # Lista de columnas a eliminar
    updated_alternative_boxes = alternative_boxes[['Hersteller',"Bezeichnung","L","B","H","Zuladung","Leergewicht","Batterietyp","Ergebnis","Batteries per box","alternative_Anzahl_Batterien","alternative_Zuladung_Auss_KG","alternative_Anzahl_benötigen_Behälter","alternative_Zuladung_kg_je_Behälter","alternative_max_Vermiculite_Gesamt_1","alternative_Anzahl_100l_Säcke_Vermiculite","alternative_Menge_Vermiculite_Behälter"]]

    
    
else:
    pass
    



#-------------------- BESCHEID PYROBUBBLES-------------------------------------------------------------------


# NOTA A VECES LAS COLUMNAS SE CREAN SIN ESPACIO O CON ESPACIO:
# Anzahl_100l_Säcke_Vermiculite vs 'Anzahl 100l Säcke Vermiculite'
# la columna Anzahl_100l_Säcke_Vermiculite  va con esta columna:  Vermiculite_Ausschöpfung_Zuladung
# Pero la columna 'Anzahl 100l Säcke Vermiculite' (no guion) va con Vermiculite Gesamt in l 

if any(boxes["Bezeichnung"].str.contains('Gen')):
    
    if 'Anzahl 100l Säcke Vermiculite' in boxes.columns:
        # La columna existe, realiza operaciones condicionales aquí
        
        # Define the replacement value
        palabra_similar = 'Pyrobubbles'
        # Function to apply the replacement
        
        def reemplazar_palabra(row):
            if 'Gen' in row['Bezeichnung']:
                return palabra_similar
            return row['Anzahl 100l Säcke Vermiculite']
        # Apply the replacement only when 'Bezeichnung' contains 'Gen'
        boxes['Anzahl 100l Säcke Vermiculite'] = boxes.apply(reemplazar_palabra, axis=1)
        boxes['Vermiculite Gesamt in l'] = boxes.apply(reemplazar_palabra, axis=1)
           
        
        pass 
    
    
    elif 'Anzahl_100l_Säcke_Vermiculite' in boxes.columns:
                # La columna existe, realiza operaciones condicionales aquí
        
        # Define the replacement value
        palabra_similar = 'Pyrobubbles'
        # Function to apply the replacement
        
        def reemplazar_palabra(row):
            if 'Gen' in row['Bezeichnung']:
                return palabra_similar
            return row['Anzahl_100l_Säcke_Vermiculite']
        # Apply the replacement only when 'Bezeichnung' contains 'Gen'
        boxes['Anzahl_100l_Säcke_Vermiculite'] = boxes.apply(reemplazar_palabra, axis=1)
        boxes['Vermiculite_Ausschöpfung_Zuladung'] = boxes.apply(reemplazar_palabra, axis=1)
        
        pass
    
    else: # La columna no existe, realiza acciones alternativas aquí
        pass
    

else:
    pass





#------------------------------------------------------------------------------------------------------------


# PASAR INFORMACION SEGUN TIPO DE BATERIA Y EN FORMATO df 

if boxes.empty:
    print("----------------------------------------------------------------------------------------------")
    print("\033[1mSorry, none of our boxes fit your dimensions or can contain the required weight.\033[0m")

else :
    print("-------------------------------------------------")
    
    #boxes = merged_df.sort_values(by='Batteries per box', ascending=True)
    
        # calcular el número de cajas necesarias
    batteries_per_box = merged_df.iloc[0]['Batteries per box']
    num_boxes = math.ceil(num_batteries / batteries_per_box)
    
    
    
    #-------------------------------- EXPORTING FILES -------------------------------------------------------------------
    # Frage den Benutzer nach dem Exporting normal file
    question = input("Remember not to have any file called Boxes in Downloads before you select Yes.,\n"
                 "Do you want to export the data as an Excel file? Yes or No: ").upper()
    
    # Überprüfe den eingegebenen Batterietyp
    if question == "YES" or question == "Y" or question == "Yes" or question == "yes" or question == "y":
        # Specify the file path and name for the Excel file
        file_path = "~/Downloads/boxes.xlsx"
        
        # Export the DataFrame to Excel
        boxes.to_excel(file_path, index=False)
        print("Excel file exported successfully.")
       
    
                #--------   
        palabra_objetivo = "max. Zuladung des Behälters wird überschritten"
        
        if any(boxes['Ergebnis'] == palabra_objetivo):
            # Frage den Benutzer nach dem exporting alternative file
            question = input("Do you want to export the 'max. Zuladung des Behälters wird überschritten' file as well? Yes or No: ").upper()
            # Überprüfe den eingegebenen Batterietyp
            
            if question == "YES" or question == "Y" or question == "Yes" or question == "yes" or question == "y":
                
                # Specify the file path and name for the Excel file
                file_path = "~/Downloads/alternative_boxes.xlsx"
                
                # Export the DataFrame to Excel
                updated_alternative_boxes.to_excel(file_path, index=False)
                print("Excel file exported successfully.")
                
            elif question == "NO" or question == "N" or question == "no" or question == "n":
                print("No export")
                
            else:
                print("Something went wrong. Please try again.")
        else:
            pass 
        
        
       
    elif question == "NO" or question == "N" or question == "no" or question == "n":
        print("No export")
        
        
                       #--------   
        palabra_objetivo = "max. Zuladung des Behälters wird überschritten"
        
        if any(boxes['Ergebnis'] == palabra_objetivo):
            # Frage den Benutzer nach dem exporting alternative file
            question = input("Do you want to export the 'max. Zuladung des Behälters wird überschritten' file as well? Yes or No: ").upper()
            # Überprüfe den eingegebenen Batterietyp
            
            if question == "YES" or question == "Y" or question == "Yes" or question == "yes" or question == "y":
                
                # Specify the file path and name for the Excel file
                file_path = "~/Downloads/alternative_boxes.xlsx"
                
                # Export the DataFrame to Excel
                updated_alternative_boxes.to_excel(file_path, index=False)
                print("Excel file exported successfully.")
                
            elif question == "NO" or question == "N" or question == "no" or question == "n":
                print("No export")
                
            else:
                print("Something went wrong. Please try again.")
    
    else:
        print("Something went wrong. Please try again.")
        
   
    
boxes


Geben Sie den gewünschten Batterietyp ein (A, B oder C): b
N° Batterien: 50
Nettogewicht Batterie: 50
Geben Sie die Höhe der Box in mm an:50
Geben Sie die Breite der Box in mm an:50
Geben Sie die Länge der Box in mm an:50


C:\Users\Stephan\AppData\Local\Temp\ipykernel_2248\2549508838.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boxes["Vermiculite_Ausschöpfung"] = ((boxes["L"]*boxes["B"]*boxes["H"]*Behälter_bei_Max_Auss)-(int(box_height)*int(box_width)*int(box_length)*num_batteries))/1000000
C:\Users\Stephan\AppData\Local\Temp\ipykernel_2248\2549508838.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boxes["Anzahl_100l_Säcke_Vermiculite"]=boxes["Vermiculite_Ausschöpfung"]/100
C:\Users\Stephan\AppData\Local\Temp\ipy

-------------------------------------------------
Remember not to have any file called Boxes in Downloads before you select Yes.,
Do you want to export the data as an Excel file? Yes or No: no
No export
Do you want to export the 'max. Zuladung des Behälters wird überschritten' file as well? Yes or No: no
No export


,Hersteller,Bezeichnung,L,B,H,Zuladung,Zuladung_KG_je_Behälter,Leergewicht,Batterietyp,Ergebnis,Anzahl_der_benötigen_Behälter,Batteries per box,Vermiculite Gesamt in l,Anzahl 100l Säcke Vermiculite
0,Holzkiste,X114,770,570,540,114,73.69,40.0,A/B,Gut aber Check,50,1,119,119
1,Holzkiste,Y390,1170,770,470,390,92.33,60.0,A/B,Gut aber Check,50,1,212,212
4,Genius,Gen LP1,2699,2000,570,2024,357.67,764.0,B/C,Gut aber Check,50,1,Pyrobubbles,Pyrobubbles
5,Gelkoh,XL+,2505,1665,430,950,229.33,1370.0,B/C,Gut aber Check,50,1,897,897
6,Europlast,Paloxe 400,1115,715,245,300,69.52,26.0,A/B,Gut aber Check,50,1,98,98
7,Europlast,Paloxe 760,1120,920,600,400,111.81,44.0,A/B,Gut aber Check,50,1,310,310
8,Zarges,AkkuSafe,430,540,130,51,53.01,11.5,B/C,max. Zuladung des Behälters wird überschritten,50,1,16,16
9,Zarges,groß,550,350,310,70,55.96,5.0,A/B,Gut aber Check,50,1,30,30
10,Zarges,klein,350,250,310,58,52.70,3.1,A/B,Gut aber Check,50,1,14,14
11,Nefab,Nefab,2354,1754,531,1850,269.23,203.0,B,Gut aber Check,50,1,1097,1097


### Calculate the number of Pyrobubbles required without reaching the maximum box weight.

In [5]:
#For PYROBUBBLES TYPE A UND TYP B < 30 kg


    
boxes["Anzahl der benötigen Behälter"]=Behälter_bei_Max_Auss
    # Calcular vermiculita para el tipo C
max_Anzahl_Batterien_Behälter = medida_a * medida_b * medida_c
boxes["Pyrobubbles Gesamt in l"] = ((boxes["L"] * boxes["B"] * boxes["H"] * boxes["Anzahl der benötigen Behälter"]) - (int(box_height) * int(box_width) * int(box_length) * num_batteries)) / 1000000     
#boxes["Anzahl 100l Säcke Vermiculite"]= boxes["Pyrobubbles Gesamt in l"] # type B > 30kg follows a different patron to calcultate the Zuladung in kg je Behälter
boxes["Anzahl 50l Pyrobubbles"] = boxes["Pyrobubbles Gesamt in l"] / 50
boxes["Anzahl 50l Pyrobubbles"] = boxes["Anzahl 50l Pyrobubbles"].apply(lambda x: math.ceil(x))
    #boxes["Batteries per box"] = num_batteries # Type C = box per battery
    #boxes.drop('Vermiculite Ausschöpfung', axis=1, inplace=True) # keep only Vermiculi for type c
#boxes.drop('Anzahl 100l Säcke Vermiculite', axis=1, inplace=True)
boxes

,Hersteller,Bezeichnung,L,B,H,Zuladung,Zuladung_KG_je_Behälter,Leergewicht,Batterietyp,Ergebnis,Anzahl_der_benötigen_Behälter,Batteries per box,Vermiculite Gesamt in l,Anzahl 100l Säcke Vermiculite,Anzahl der benötigen Behälter,Pyrobubbles Gesamt in l,Anzahl 50l Pyrobubbles
0,Holzkiste,X114,770,570,540,114,119.50,40.0,A/B,max. Zuladung des Behälters wird überschritten,50,1,98,98,25,3825.150000,77
1,Holzkiste,Y390,1170,770,470,390,138.14,60.0,A/B,Gut aber Check,50,1,191,191,9,1710.807000,35
4,Genius,Gen LP1,2699,2000,570,2024,403.49,764.0,B/C,Gut aber Check,50,1,Pyrobubbles,Pyrobubbles,2,4053.720000,82
5,Gelkoh,XL+,2505,1665,430,950,275.15,1370.0,B/C,Gut aber Check,50,1,876,876,2,1486.909500,30
7,Europlast,Paloxe 760,1120,920,600,400,157.62,44.0,A/B,Gut aber Check,50,1,289,289,9,3464.160000,70
11,Nefab,Nefab,2354,1754,531,1850,315.05,203.0,B,Gut aber Check,50,1,1076,1076,2,2284.908792,46
13,BB Verp,GG-Karton 570,570,370,430,60,104.87,60.0,A/B,max. Zuladung des Behälters wird überschritten,50,1,25,25,50,2434.350000,49
14,Karton,GG-Karton 770,770,570,550,150,119.94,150.0,A/B,Gut aber Check,50,1,100,100,25,3934.875000,79


In [7]:
#For PYROBUBBLES TYPE B < 30kg AND c


    
boxes["Anzahl der benötigen Behälter"]=num_batteries
    # Calcular vermiculita para el tipo C
max_Anzahl_Batterien_Behälter = medida_a * medida_b * medida_c
boxes["Pyrobubbles Gesamt in l"] = ((boxes["L"] * boxes["B"] * boxes["H"] * boxes["Anzahl der benötigen Behälter"]) - (int(box_height) * int(box_width) * int(box_length) * num_batteries)) / 1000000     
#boxes["Anzahl 100l Säcke Vermiculite"]= boxes["Pyrobubbles Gesamt in l"] # type B > 30kg follows a different patron to calcultate the Zuladung in kg je Behälter
boxes["Anzahl 50l Pyrobubbles"] = boxes["Pyrobubbles Gesamt in l"] / 50
boxes["Anzahl 50l Pyrobubbles"] = boxes["Anzahl 50l Pyrobubbles"].apply(lambda x: math.ceil(x))
    #boxes["Batteries per box"] = num_batteries # Type C = box per battery
    #boxes.drop('Vermiculite Ausschöpfung', axis=1, inplace=True) # keep only Vermiculi for type c
#boxes.drop('Anzahl 100l Säcke Vermiculite', axis=1, inplace=True)
boxes

,Hersteller,Bezeichnung,L,B,H,Zuladung,Zuladung_KG_je_Behälter,Leergewicht,Batterietyp,Ergebnis,Anzahl_der_benötigen_Behälter,Batteries per box,Vermiculite Gesamt in l,Anzahl 100l Säcke Vermiculite,Anzahl der benötigen Behälter,Pyrobubbles Gesamt in l,Anzahl 50l Pyrobubbles
0,Holzkiste,X114,770,570,540,114,119.50,40.0,A/B,max. Zuladung des Behälters wird überschritten,50,1,98,98,50,9750.3000,196
1,Holzkiste,Y390,1170,770,470,390,138.14,60.0,A/B,Gut aber Check,50,1,191,191,50,19071.1500,382
4,Genius,Gen LP1,2699,2000,570,2024,403.49,764.0,B/C,Gut aber Check,50,1,Pyrobubbles,Pyrobubbles,50,151743.0000,3035
5,Gelkoh,XL+,2505,1665,430,950,275.15,1370.0,B/C,Gut aber Check,50,1,876,876,50,87572.7375,1752
7,Europlast,Paloxe 760,1120,920,600,400,157.62,44.0,A/B,Gut aber Check,50,1,289,289,50,28812.0000,577
11,Nefab,Nefab,2354,1754,531,1850,315.05,203.0,B,Gut aber Check,50,1,1076,1076,50,107522.7198,2151
13,BB Verp,GG-Karton 570,570,370,430,60,104.87,60.0,A/B,max. Zuladung des Behälters wird überschritten,50,1,25,25,50,2434.3500,49
14,Karton,GG-Karton 770,770,570,550,150,119.94,150.0,A/B,Gut aber Check,50,1,100,100,50,9969.7500,200


## New DF with data on the amount of Pyrobubbles required.

In [22]:

if 'Anzahl 100l Säcke Vermiculite' in boxes.columns:
    
    if (battery_type == "B" and battery_weight >= 30) or battery_type == "C":
        #PARA PYROBUBBLES TYP A UND TYP B < 30 and C
        boxes["Anzahl der benötigen Behälter"]=num_batteries
        # Calcular vermiculita para el tipo C
        max_Anzahl_Batterien_Behälter = medida_a * medida_b * medida_c
        boxes["Pyrobubbles Gesamt in l"] = ((boxes["L"] * boxes["B"] * boxes["H"] * boxes["Anzahl_der_benötigen_Behälter"]) - (int(box_height) * int(box_width) * int(box_length) * num_batteries)) / 1000000     
        #boxes["Anzahl 100l Säcke Vermiculite"]= boxes["Pyrobubbles Gesamt in l"] # type B > 30kg follows a different patron to calcultate the Zuladung in kg je Behälter
        boxes["Anzahl 50l Pyrobubbles"] = boxes["Pyrobubbles Gesamt in l"] / 50
        boxes["Anzahl 50l Pyrobubbles"] = boxes["Anzahl 50l Pyrobubbles"].apply(lambda x: math.ceil(x))
            #boxes["Batteries per box"] = num_batteries # Type C = box per battery
            #boxes.drop('Vermiculite Ausschöpfung', axis=1, inplace=True) # keep only Vermiculi for type c
            #boxes.drop('Anzahl 100l Säcke Vermiculite', axis=1, inplace=True)
            
        boxes.loc[boxes['Anzahl 100l Säcke Vermiculite'] != 'Pyrobubbles', 'Anzahl 50l Pyrobubbles'] = 'Vermiculite'
        boxes.loc[boxes['Anzahl 100l Säcke Vermiculite'] != 'Pyrobubbles', 'Pyrobubbles Gesamt in l'] = 'Vermiculite'
            
    elif battery_type == "B" and battery_weight < 30:
        max_Anzahl_Batterien_Behälter = medida_a * medida_b * medida_c
        boxes["Pyrobubbles Gesamt in l"] = ((boxes["L"] * boxes["B"] * boxes["H"] * boxes["Anzahl_der_benötigen_Behälter"]) - (int(box_height) * int(box_width) * int(box_length) * num_batteries)) / 1000000     
        #boxes["Anzahl 100l Säcke Vermiculite"]= boxes["Pyrobubbles Gesamt in l"] # type B > 30kg follows a different patron to calcultate the Zuladung in kg je Behälter
        boxes["Anzahl 50l Pyrobubbles"] = boxes["Pyrobubbles Gesamt in l"] / 50
        boxes["Anzahl 50l Pyrobubbles"] = boxes["Anzahl 50l Pyrobubbles"].apply(lambda x: math.ceil(x))
            #boxes["Batteries per box"] = num_batteries # Type C = box per battery
            #boxes.drop('Vermiculite Ausschöpfung', axis=1, inplace=True) # keep only Vermiculi for type c
            #boxes.drop('Anzahl 100l Säcke Vermiculite', axis=1, inplace=True)
            
            
        boxes.loc[boxes['Anzahl 100l Säcke Vermiculite'] != 'Pyrobubbles', 'Anzahl 50l Pyrobubbles'] = 'Vermiculite'
        boxes.loc[boxes['Anzahl 100l Säcke Vermiculite'] != 'Pyrobubbles', 'Pyrobubbles Gesamt in l'] = 'Vermiculite'  
            
            
elif 'Anzahl_100l_Säcke_Vermiculite' in boxes.columns: 
    
    if (battery_type == "B" and battery_weight >= 30) or battery_type == "C":
        #PARA PYROBUBBLES TYP A UND TYP B < 30 and C
            boxes["Anzahl der benötigen Behälter"]=num_batteries
            # Calcular vermiculita para el tipo C
            max_Anzahl_Batterien_Behälter = medida_a * medida_b * medida_c
            boxes["Pyrobubbles Gesamt in l"] = ((boxes["L"] * boxes["B"] * boxes["H"] * boxes["Anzahl_der_benötigen_Behälter"]) - (int(box_height) * int(box_width) * int(box_length) * num_batteries)) / 1000000     
            #boxes["Anzahl 100l Säcke Vermiculite"]= boxes["Pyrobubbles Gesamt in l"] # type B > 30kg follows a different patron to calcultate the Zuladung in kg je Behälter
            boxes["Anzahl 50l Pyrobubbles"] = boxes["Pyrobubbles Gesamt in l"] / 50
            boxes["Anzahl 50l Pyrobubbles"] = boxes["Anzahl 50l Pyrobubbles"].apply(lambda x: math.ceil(x))
            #boxes["Batteries per box"] = num_batteries # Type C = box per battery
            #boxes.drop('Vermiculite Ausschöpfung', axis=1, inplace=True) # keep only Vermiculi for type c
            #boxes.drop('Anzahl 100l Säcke Vermiculite', axis=1, inplace=True)

            
            boxes.loc[boxes['Anzahl_100l_Säcke_Vermiculite'] != 'Pyrobubbles', 'Anzahl 50l Pyrobubbles'] = 'Vermiculite'
            boxes.loc[boxes['Anzahl_100l_Säcke_Vermiculite'] != 'Pyrobubbles', 'Pyrobubbles Gesamt in l'] = 'Vermiculite'

    elif battery_type == 'B' and battery_weight < 30:
        
        
        boxes["Anzahl der benötigen Behälter"]=Behälter_bei_Max_Auss
            # Calcular vermiculita para el tipo C
        max_Anzahl_Batterien_Behälter = medida_a * medida_b * medida_c
        boxes["Pyrobubbles Gesamt in l"] = ((boxes["L"] * boxes["B"] * boxes["H"] * boxes["Anzahl_der_benötigen_Behälter"]) - (int(box_height) * int(box_width) * int(box_length) * num_batteries)) / 1000000     
            #boxes["Anzahl 100l Säcke Vermiculite"]= boxes["Pyrobubbles Gesamt in l"] # type B > 30kg follows a different patron to calcultate the Zuladung in kg je Behälter
        boxes["Anzahl 50l Pyrobubbles"] = boxes["Pyrobubbles Gesamt in l"] / 50
        boxes["Anzahl 50l Pyrobubbles"] = boxes["Anzahl 50l Pyrobubbles"].apply(lambda x: math.ceil(x))
            #boxes["Batteries per box"] = num_batteries # Type C = box per battery
            #boxes.drop('Vermiculite Ausschöpfung', axis=1, inplace=True) # keep only Vermiculi for type c
            #boxes.drop('Anzahl 100l Säcke Vermiculite', axis=1, inplace=True)  
            

        # reaplacing and marking data     
        boxes.loc[boxes['Anzahl_100l_Säcke_Vermiculite'] != 'Pyrobubbles', 'Anzahl 50l Pyrobubbles'] = 'Vermiculite'
        boxes.loc[boxes['Anzahl_100l_Säcke_Vermiculite'] != 'Pyrobubbles', 'Pyrobubbles Gesamt in l'] = 'Vermiculite'
            
else: 
    
    print("Check the Pyrobubbles code & Anzahl_100l VS Anzahl 100l")
boxes

,Hersteller,Bezeichnung,L,B,H,Zuladung,Zuladung_KG_je_Behälter,Leergewicht,Batterietyp,Ergebnis,Anzahl_der_benötigen_Behälter,Batteries per box,Vermiculite Gesamt in l,Anzahl 100l Säcke Vermiculite,Anzahl der benötigen Behälter,Pyrobubbles Gesamt in l,Anzahl 50l Pyrobubbles
0,Holzkiste,X114,770,570,540,114,73.69,40.0,A/B,Gut aber Check,50,1,119,119,50,Vermiculite,Vermiculite
1,Holzkiste,Y390,1170,770,470,390,92.33,60.0,A/B,Gut aber Check,50,1,212,212,50,Vermiculite,Vermiculite
4,Genius,Gen LP1,2699,2000,570,2024,357.67,764.0,B/C,Gut aber Check,50,1,Pyrobubbles,Pyrobubbles,50,153836.75,3077
5,Gelkoh,XL+,2505,1665,430,950,229.33,1370.0,B/C,Gut aber Check,50,1,897,897,50,Vermiculite,Vermiculite
6,Europlast,Paloxe 400,1115,715,245,300,69.52,26.0,A/B,Gut aber Check,50,1,98,98,50,Vermiculite,Vermiculite
7,Europlast,Paloxe 760,1120,920,600,400,111.81,44.0,A/B,Gut aber Check,50,1,310,310,50,Vermiculite,Vermiculite
8,Zarges,AkkuSafe,430,540,130,51,53.01,11.5,B/C,max. Zuladung des Behälters wird überschritten,50,1,16,16,50,Vermiculite,Vermiculite
9,Zarges,groß,550,350,310,70,55.96,5.0,A/B,Gut aber Check,50,1,30,30,50,Vermiculite,Vermiculite
10,Zarges,klein,350,250,310,58,52.70,3.1,A/B,Gut aber Check,50,1,14,14,50,Vermiculite,Vermiculite
11,Nefab,Nefab,2354,1754,531,1850,269.23,203.0,B,Gut aber Check,50,1,1097,1097,50,Vermiculite,Vermiculite
